In [2]:
import os
import numpy as np, pandas as pd
import keras

import torch
from torch import nn, optim
from torchvision import datasets, models, transforms

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'keras'

In [ ]:
data_directory = '/content/drive/MyDrive/Dogs_Dataset/images_cropped'

In [ ]:
true_labels = []
k = 0
for i in os.listdir(data_directory):
  for j in os.listdir(os.path.join(data_directory,i)):
    true_labels.append(k)
  k+=1
true_labels = np.array(true_labels)

**Resize**

In [ ]:
transforms = transforms.Compose([
                              transforms.Resize(224),
                              transforms.ToTensor(),
                             ])

In [ ]:
image_datasets = datasets.ImageFolder(data_directory, transform=transforms)

In [ ]:
dataloaders  = torch.utils.data.DataLoader(image_datasets, batch_size=1, shuffle = False)

**Normalize**

In [ ]:
norm_data = []
for i in dataloaders:
  mean,std,var = torch.mean(i[0]),torch.std(i[0]),torch.var(i[0])
  t = (i[0]-mean)/std
  norm_data.append(t)

**Extract Features**

In [ ]:
resnet18 = models.resnet18(pretrained=True)
resnet18 = torch.nn.Sequential(*(list(resnet18.children())[:-1]))

resnet18.eval()

In [ ]:
features_extracted = []
for i in norm_data:
  with torch.no_grad():
    features = resnet18(i)
  features_extracted.append(features)

**Dimension Reduction**

In [ ]:
features_array = []
for i in features_extracted:
  output_tensor = torch.nn.functional.adaptive_avg_pool2d(i, (1, 1))
  output_tensor = output_tensor.view(512)
  output_tensor = np.array(output_tensor)
  features_array.append(output_tensor)

In [ ]:
features_array = np.array(features_array)
features_array.shape

**Clustering**

KMeans

In [ ]:
from sklearn.cluster import KMeans
kmeans_1 = KMeans(n_clusters=4, random_state=0, init="random", n_init = 'auto').fit(features_array)
kmeans_2 = KMeans(n_clusters=4, random_state=0, init="k-means++", n_init = 'auto').fit(features_array)

In [ ]:
from sklearn.cluster import BisectingKMeans
kmeans_3 = BisectingKMeans(n_clusters=4,init='random').fit(features_array)

In [ ]:
from sklearn.cluster import SpectralClustering
spectral = SpectralClustering(n_clusters=4).fit(features_array)

**DBSCAN**

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=7, min_samples=2).fit(features_array)
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print("Number of clusters:",n_clusters_)

For eps = 7 and min_samples = 2, n_clusters = 4

**Agglomerative Clustering**

In [ ]:
from sklearn.cluster import AgglomerativeClustering
ag_single = AgglomerativeClustering(n_clusters=4, linkage = 'single').fit(features_array)
ag_complete = AgglomerativeClustering(n_clusters=4, linkage = 'complete').fit(features_array)
ag_group = AgglomerativeClustering(n_clusters=4, linkage = 'average').fit(features_array)
ag_ward = AgglomerativeClustering(n_clusters=4, linkage = 'ward').fit(features_array)

**Clustering Evaluation**

In [ ]:
cluster_models = [kmeans_1,kmeans_2,kmeans_3,spectral,db,ag_single,ag_complete,ag_group,ag_ward]
fms = {'KMeans-Random':0,
       'KMeans-kmeans++':0,
       'Bisecting KMeans':0,
       'Spectral Clustering':0,
       'DBSCAN':0,
       'Agglomerative Clustering - single':0,
       'Agglomerative Clustering - complete':0,
       'Agglomerative Clustering - average':0,
       'Agglomerative Clustering - ward':0}
ss = {'KMeans-Random':0,
       'KMeans-kmeans++':0,
       'Bisecting KMeans':0,
       'Spectral Clustering':0,
       'DBSCAN':0,
       'Agglomerative Clustering - single':0,
       'Agglomerative Clustering - complete':0,
       'Agglomerative Clustering - average':0,
       'Agglomerative Clustering - ward':0}

In [ ]:
from sklearn.metrics import fowlkes_mallows_score
from sklearn.metrics import silhouette_score
for i,j,k in zip(cluster_models,fms,ss):
  predicts = i.fit_predict(features_array)
  fm_score = fowlkes_mallows_score(true_labels,predicts)
  s_score = silhouette_score(np.array(true_labels).reshape(-1,1),predicts)
  fms[j] = fm_score
  ss[k] = s_score

**Ranking best to worst on fowlkes-mallows_score**

In [ ]:
sorted(fms.items(), key=lambda item: item[1])[::-1]

**Ranking best to worst on Silhouette score**

In [ ]:
sorted(ss.items(), key=lambda item: item[1])[::-1]